### Essay Vectorize

In [5]:
import numpy as np
import re
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import csv

In [ ]:
# 셀에서 출력 정확도 설정.
%precision %.3f

In [ ]:
file_path = r'.\crawling_data\merged_good_translated_essay.csv'

# 자소서 리스트화
documents = []
with open(file_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        documents.append(row[0])

In [ ]:
# 불용어 리스트 가져오기
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

In [ ]:
documents_corpus = []
for i in range(len(documents)):
    document = preprocess(documents[i])
    document = re.sub(r'\W', ' ', document)
    document = re.sub(r'_', ' ', document)
    document = re.sub(r'^br$', ' ', document)
    document = re.sub(r'\s+br\s+',' ',document)
    document = re.sub(r'\s+[a-z]\s+', ' ',document)
    document = re.sub(r'^b\s+', '', document)
    document = re.sub(r'\s+', ' ', document)               # 잉여 space 제거.
    document = re.sub(",|\n|@|:", "", document)            # 쉼표, \n, @ 제거
    document = re.sub(r'\([^)]*\)', '', document)          # 소괄호 제거
    documents_corpus.append(document)

In [ ]:
# Tagged Corpus 생성.
# 모델 학습에 필요한 형태로 변환.
my_tagged_corpus = [TaggedDocument(words=my_words, tags=[i]) for i, my_words in enumerate(documents_corpus)]

In [ ]:
# Doc2Vec 모델 생성과 준비.
my_model = Doc2Vec(
    window=5,                # 문맥을 고려하는 단어의 최대 거리.
    vector_size=130,         # 벡터 크기
    alpha=0.025,             # 모델의 학습속도.
    min_alpha=0.025,
    workers = 8,             # 학습을 병렬화하기 위한 스레드 수.
    min_count=3,             # 최소 단어 빈도.
    dm = 0,                  # dm = '0'이면 DBOW. dm = '1' 이면 DM.
    negative = 6,            # Negative sampling 수.
    seed = 1234)             # 난수 seed.

# 모델 학습 전 필요한 단계.
my_model.build_vocab(my_tagged_corpus)

In [ ]:
# Doc2Vec 모델 학습.
max_epochs = 11
for epoch in range(max_epochs):
    print('Iteration #{}'.format(epoch))
    my_model.train(my_tagged_corpus,
                total_examples=len(my_corpus),
                epochs=10)
    my_model.alpha -= 0.002                  # 학습 속도 점진적 감소.
    my_model.min_alpha = my_model.alpha
my_model.save('my_model.d2v')